In [3]:
import os
import sys
import numpy as np
from multiprocessing.pool import ThreadPool
import subprocess


def generateStep3Data(window_size):
    proc_cmd = "Rscript Step3_SensorData_Feature_Engineering.R %s" % str(window_size)

    proc = subprocess.Popen(proc_cmd, shell=True, stdout=subprocess.PIPE)

    out = proc.stdout.read()

    proc.kill()

    return out


output_list = []


def addToOutput(out):
    output_list.append(out)


if __name__ == '__main__':

    cwd = os.getcwd()

    if os.path.isdir("../Rscripts"):
        os.chdir("../Rscripts")
    else:
        print("../Rscripts folder doesn't exist, please check again")
        sys.exit(0)

    # Define sample frequency in current dataset
    sample_freq = 240
    window_size_list = list(np.arange(0.5, 5.0, 0.5))

    th_pool = ThreadPool(processes=len(window_size_list))

    for p in window_size_list:
        window_size = int(sample_freq * p)

        async_result = th_pool.apply_async(func=generateStep3Data, args=[window_size], callback=addToOutput)

    th_pool.close()
    th_pool.join()
    
    # Recover working directory
    os.chdir(cwd)
